In [1]:
# LIU Jinghui
# 20745644

import os
import numpy as np
import pandas as pd
from time import time
import datetime

# ---
# data prepare
# ---
# train: 1961 -- 1978
# validation: 1979 -- 1990
# test: 1991 -- 2020

# train_data = all_data[all_data.DATE <= 19781231][19601231 < all_data.DATE]
# validation_data = all_data[19781231 < all_data.DATE][all_data.DATE <= 19901231]
# test_data = all_data[all_data.DATE > 19911231]

import os
import numpy as np
import pandas as pd
from time import time
import datetime

from scipy import stats
import statsmodels.api as sm
from statsmodels import regression
import pickle

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn import metrics
from sklearn.impute import SimpleImputer

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectFromModel

from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import GradientBoostingRegressor as GBR

from six import StringIO
import seaborn as sns

In [2]:
# ---
# initialization
# ---
data_path = 'C:\\Users\\Claudia\\Desktop\\philostudy\\hkust\\21-22 term\\6010Z\\project2-1112\\DATA\\DATA\\'
model_path = 'C:\\Users\\Claudia\\Desktop\\philostudy\\hkust\\21-22 term\\6010Z\\project2-1112\\MODEL\\'

# train_data = pd.read_csv(data_path+'train_data.csv',index_col=0)
# validation_data= pd.read_csv(data_path+'validation_data.csv',index_col=0)

In [ ]:
# ---
# GBR
# ---
loss_list = ['huber'] #, 'squared_error']
learning_rate = [0.01, 0.03, 0.05, 0.1]
max_depth = [3,4,5,6]

In [1]:
starttime = datetime.datetime.now()

v_R = dict()

for i in range(1,31):
    print(i)
    test = pd.read_csv(data_path+'test_'+str(i)+'.csv',index_col=0)
    
    # ---
    # preprocess
    # ---
    check_nan_train = ((train_data.isnull().sum())/train_data.shape[0]).sort_values(ascending=False)
    check_nan_validation = ((validation_data.isnull().sum())/validation_data.shape[0]).sort_values(ascending=False)
    
    train_ = preprocess(train_data,check_nan_train)
    x_train = train_.drop(['permno','DATE','RET'],axis=1)
    y_train = train_.RET
    del train_
    
    validation_ = preprocess(validation_data,check_nan_validation)
    x_validate = validation_.drop(['permno','DATE','RET'],axis=1)
    y_validate = validation_.RET
    del validation_
    
    print('finish preprocess!')
    
    for l in loss_list:
        for lr in learning_rate:
            for max_dep in max_depth:
                time0 = datetime.datetime.now()
                print('start training:', time0)
                
                # ---
                # model training
                # ---                
                gbr = GBR(max_depth=max_dep,learning_rate=lr,loss=l)
                gbr.fit(x_train,y_train)
                
                train_time = datetime.datetime.now()
                print('training time：',int((train_time - time0).seconds/60),'min')
                
                # model saving
                modelName = str(i) + '_' + l + '_' + str(lr) + '_'+ str(max_dep)
                modelFile = model_path + modelName + '.sav'
                pickle.dump(gbr, open(modelFile, 'wb'))
    
                # ---
                # validation
                # ---
                validate_R_oos = R_oos(gbr,x_validate,y_validate)
                v_R[modelName] = validate_R_oos
                del gbr
                
                validation_time = datetime.datetime.now()
                print('validating time：',int((validation_time - train_time).seconds/60),'min')
    
    best_model = min(v_R, key=v_R.get)
    
    # ---
    # load model
    # ---
    #loaded_model = pickle.load(open(model_path + best_model  '.sav', 'rb'))
    
    # ---
    # test
    # ---
    x_test = test.drop(['permno','DATE','RET'],axis=1)
    y_test = test.RET
    del test
    
    test_R_oos = R_oos(x_test, y_test)
    print(test_R_oos)
    
    # update data
    train_end = train_data.DATE.iloc[-1]+10
    mid =  validation_data[validation_data.DATE <= (train_end+10000)]
    train_data = train_data.append(mid)
    validation_data = validation_data[validation_data.DATE > (train_end+10000)]
    validation_data = validation_data.append(test)
    
    i += 1
    
    del test
    
endtime = datetime.datetime.now()
print ('运行时长：',int((endtime - starttime).seconds/60),'分钟') 

In [ ]:
# ---
# RFR
# ---
n_estimators = [300,400,500,600]
max_depth = [3,4,5,6]
min_samples_split = [20,50,100,200]

In [ ]:
starttime = datetime.datetime.now()
v_R = dict()

for i in range(1,31):
    print(i)
    test = pd.read_csv(data_path+'test_'+str(i)+'.csv',index_col=0)
    
    # ---
    # preprocess
    # ---
    check_nan_train = ((train_data.isnull().sum())/train_data.shape[0]).sort_values(ascending=False)
    check_nan_validation = ((validation_data.isnull().sum())/validation_data.shape[0]).sort_values(ascending=False)
    
    train_ = preprocess(train_data,check_nan_train)
    x_train = train_.drop(['permno','DATE','RET'],axis=1)
    y_train = train_.RET
    del train_
    
    validation_ = preprocess(validation_data,check_nan_validation)
    x_validate = validation_.drop(['permno','DATE','RET'],axis=1)
    y_validate = validation_.RET
    del validation_
    
    print('finish preprocess!')
    
    for n in n_estimators:
        for s in min_samples_split:
            for max_dep in max_depth:
                
                time0 = datetime.datetime.now()
                # ---
                # model training
                # ---                
                rfr = RFR(n_estimators=n,min_samples_split=s,max_depth=max_dep)
                rfr.fit(x_train,y_train)
                
                train_time = datetime.datetime.now()
                print('training time：',int((train_time - time0).seconds/60),'min')
                
                # model saving
                modelName = str(n) + '_' + str(s) + '_'+ str(max_dep)
                modelFile = model_path + modelName + '.sav'
                pickle.dump(rfr, open(modelFile, 'wb'))
    
                # ---
                # validation
                # ---
                validate_R_oos = R_oos(rfr,x_validate,y_validate)
                k = 'test_'+str(i)+modelName
                v_R[k] = validate_R_oos
                del rfr
                
                validation_time = datetime.datetime.now()
                print('validating time：',int((validation_time - train_time).seconds/60),'min')
    
    best_model = min(v_R, key=v_R.get)
    
    # ---
    # load model
    # ---
    # loaded_model = pickle.load(open(model_path + best_model  '.sav', 'rb'))
    
    # ---
    # test
    # ---
    x_test = test.drop(['permno','DATE','RET'],axis=1)
    y_test = test.RET
    del test
    
    test_R_oos = R_oos(x_test, y_test)
    print(test_R_oos)
    
    # update data
    train_end = train_data.DATE.iloc[-1]+10
    mid =  validation_data[validation_data.DATE <= (train_end+10000)]
    train_data = train_data.append(mid)
    validation_data = validation_data[validation_data.DATE > (train_end+10000)]
    validation_data = validation_data.append(test)
    
    i += 1
    
    del test
    
endtime = datetime.datetime.now()
print ('running time：',int((endtime - starttime).seconds/60),'min') 

In [5]:
def R_oos(model,x,y):
    y_hat = model.predict(x)
    fenmu = sum(y**2)
    fenzi = sum((y-y_hat)**2)
    return 1-(fenzi/fenmu)

In [4]:
from sklearn.impute import SimpleImputer

def preprocess(df, check_nan):
    #均值：
    df_mean = SimpleImputer(missing_values=np.nan, strategy='mean',copy=False)
    #中位数：
    df_median = SimpleImputer(missing_values=np.nan, strategy='median',copy=False)
    #常数0：
    df_0 = SimpleImputer(strategy="constant",fill_value=0,copy=False)
    #众数：
    df_most_frequent = SimpleImputer(missing_values=np.nan, strategy='most_frequent',copy=False)
    
    # missing rate < 50% ---- fill median
    # missing rate > 50% ---- fill 0
    available_factors = check_nan[check_nan < 0.5].index
    unavailable_factors = check_nan[check_nan >= 0.5].index
    
    ava_df = df[available_factors]
    a = df_most_frequent.fit_transform(ava_df)
    a = pd.DataFrame(a)
    a.columns = ava_df.columns
    
    un_df = df[unavailable_factors]
    u = df_0.fit_transform(un_df)
    u = pd.DataFrame(u)
    u.columns = un_df.columns
    
    df = pd.concat([a,u],axis=1)
    
    return df
